# WED SCRAPING

## Khởi Tạo

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures

## MAIN

In [3]:
pages = []
goc='https://tiki.vn/search?q=ti%E1%BB%83u+thuy%E1%BA%BFt&sort=top_seller&page=' 
def get_links(trang):
    for i in range(1,trang+1):
        pages.append(goc+str(i))
get_links(50)

In [4]:
lon=[]
lop=[]
loun=[]
sort = []
dem = 0
def take_name_and_prices(url):
    html=urlopen(url) 
    bs = BeautifulSoup(html.read(), 'html.parser') 
    

    if url[len(url)-2] != '=':
        c = url[len(url)-2]+url[len(url)-1]
    else :
        c ='0'+ url[len(url)-1]
    for i in range(1,40+1):
        if i < 10:
            temp = '0'+str(i)
        else : 
            temp = str(i)
        sort.append(c+temp)

        
    tam = bs.find_all('div',class_='name')    
    for i in tam :
        lon.append(i.text)
            
    product_prices=bs.find_all('div',class_='price-discount__price')
        
    for element in product_prices:
        lop.append(element.text)
        
    site='https://tiki.vn'
    lou=[]
        
    urls=bs.find_all('a', class_='product-item')    
    for url in urls:
        lou.append(url['href']) 
    for url in lou:    
        urlnew='{}{}'.format(site, url)
        loun.append(urlnew)
    return 1

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    results = executor.map(take_name_and_prices, pages)


In [5]:
for i in range(0,2000):
    tam = sort[i] + loun[i]
    loun[i]=tam

IndexError: list index out of range

In [14]:
sort2= []
img = []
loa=[]
genre=[]
def take_picture_info_genre(url):    
    try:
        urlnew = url[4:]
        html = urlopen(urlnew)
        soul = BeautifulSoup(html.read(), 'html.parser')
        
        try:
            sort2.append(url[0:4])
        except:
            sort2.append(url[0:4])
            
        try:
            tam = soul.find('div',class_='ToggleContent__View-sc-1dbmfaw-0 wyACs')
            loa.append(tam.text)
        except:
            loa.append('')
        try:
            s=soul.find('picture',class_='webpimg-container')
            ptd=s.find('img')
            img.append(ptd['src'])
        except:
            img.append('')

        try:
            temp = []
            for ptd in soul.find_all('a', class_ ='breadcrumb-item'):
                temp.append(ptd.text)
            genre.append(temp)
        except:
            genre.append('')
    except : 
        loa.append('')
        img.append('')
        genre.append('')
        sort2.append(url[0]+url[1]+url[2]+url[3])
    return 1

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    results = executor.map(take_picture_info_genre, loun)

In [15]:
for i in range(0,2000):
    for j in range(0,i):
        if (sort[i]<sort[j]):
            sort[i],sort[j]=sort[j],sort[i]
            lon[i],lon[j]=lon[j],lon[i]
            lop[i],lop[j]=lop[j],lop[i]
for i in range(0,2000):
    for j in range(0,i):
        if (sort2[i]<sort2[j]):
            sort2[i],sort2[j]=sort2[j],sort2[i]
            img[i],img[j]=img[j],img[i]
            loa[i],loa[j]=loa[j],loa[i]
            genre[i],genre[j]=genre[j],genre[i]

In [7]:
dict_pro={"Tên ": lon,"Gia ": lop,"Thong Tin":loa,"Ảnh ":img,"genre":genre}

In [8]:
df1=pd.DataFrame(dict_pro)

In [9]:
df1.to_csv("lgh1.csv")

## CODE CŨ (CHẬM VCL)

In [11]:
lon=[]
lop=[]
loun=[]
loa=[]
img = []
genre = []
def Webscraping(trang):
    goc='https://tiki.vn/search?q=ti%E1%BB%83u+thuy%E1%BA%BFt&sort=top_seller&page=' 
    for i in range(1,trang+1): 
        html=urlopen(goc+str(i)) 
        bs = BeautifulSoup(html.read(), 'html.parser') 
        tam = bs.find_all('div',class_='name')
        for i in tam :
            lon.append(i.text)
            
        product_prices=bs.find_all('div',class_='price-discount__price')
        
        for element in product_prices:
            lop.append(element.text)
        
        site='https://tiki.vn'
        lou=[]
        
        urls=bs.find_all('a', class_='product-item')    
        for url in urls:
            lou.append(url['href']) 
        for url in lou:    
            urlnew='{}{}'.format(site, url)
            loun.append(urlnew)
            html = urlopen(urlnew)
            soul = BeautifulSoup(html.read(), 'html.parser')
            try:
                tam = soul.find('div',class_='ToggleContent__View-sc-1dbmfaw-0 wyACs')
                loa.append(tam.text)
            except : 
                loa.append('')

            try :
                s=soul.find('picture',class_='webpimg-container')
                ptd=s.find('img')
                img.append(ptd['src'])
            except:
                img.append('')
            try:
                temp = []
                for ptd in soul.find_all('a', class_ ='breadcrumb-item'):
                    temp.append(ptd.text)
                genre.append(temp)
            except:
                genre.append('')
        
    dict_pro={"Tên ": lon,"Gia ": lop ,"Thong tin": loa,"Anh":img,"The loai":genre}
    return dict_pro

df=pd.DataFrame(Webscraping(1))

In [10]:
df.to_csv("lgh.csv")